<a href="https://colab.research.google.com/github/nguyenhaphan1/ID-recognition-with-CRNN-/blob/main/DigitRecognitionWithCRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Clone pre-implemented CRNN model from git
!git clone https://github.com/FLming/CRNN.tf2

In [ ]:
!pip install -r requirements.txt

In [3]:
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from google.colab.patches import cv2_imshow
import numpy as np

In [4]:
#define loss and accuracy class for loading the model
class CTCLoss(keras.losses.Loss):
    """A class that wraps the function of tf.nn.ctc_loss.

    Attributes:
        logits_time_major: If False (default) , shape is [batch, time, logits],
            If True, logits is shaped [time, batch, logits].
        blank_index: Set the class index to use for the blank label. default is
            -1 (num_classes - 1).
    """

    def __init__(
        self, logits_time_major=False, blank_index=-1, reduction=keras.losses.Reduction.AUTO, name="ctc_loss"
    ):
        super().__init__(name=name, reduction=reduction)
        self.logits_time_major = logits_time_major
        self.blank_index = blank_index

    def call(self, y_true, y_pred):
        """Computes CTC (Connectionist Temporal Classification) loss. Works on
        CPU, because y_true is a SparseTensor.
        """
        y_true = tf.cast(y_true, tf.int32)
        y_pred_shape = tf.shape(y_pred)
        logit_length = tf.fill([y_pred_shape[0]], y_pred_shape[1])
        loss = tf.nn.ctc_loss(
            labels=y_true,
            logits=y_pred,
            label_length=None,
            logit_length=logit_length,
            logits_time_major=self.logits_time_major,
            blank_index=self.blank_index,
        )
        return tf.math.reduce_mean(loss)

class SequenceAccuracy(keras.metrics.Metric):
    def __init__(self, name="sequence_accuracy", **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        def sparse2dense(tensor, shape):
            tensor = tf.sparse.reset_shape(tensor, shape)
            tensor = tf.sparse.to_dense(tensor, default_value=-1)
            tensor = tf.cast(tensor, tf.float32)
            return tensor

        y_true_shape = tf.shape(y_true)
        batch_size = y_true_shape[0]
        y_pred_shape = tf.shape(y_pred)
        max_width = tf.math.maximum(y_true_shape[1], y_pred_shape[1])
        logit_length = tf.fill([batch_size], y_pred_shape[1])
        decoded, _ = tf.nn.ctc_greedy_decoder(
            inputs=tf.transpose(y_pred, perm=[1, 0, 2]),
            sequence_length=logit_length,
        )
        y_true = sparse2dense(y_true, [batch_size, max_width])
        y_pred = sparse2dense(decoded[0], [batch_size, max_width])
        num_errors = tf.math.reduce_any(
            tf.math.not_equal(y_true, y_pred), axis=1
        )
        num_errors = tf.cast(num_errors, tf.float32)
        num_errors = tf.math.reduce_sum(num_errors)
        batch_size = tf.cast(batch_size, tf.float32)
        self.total.assign_add(batch_size)
        self.count.assign_add(batch_size - num_errors)

    def result(self):
        return self.count / self.total

    def reset_states(self):
        self.count.assign(0)
        self.total.assign(0)


class EditDistance(keras.metrics.Metric):
    def __init__(self, name="edit_distance", **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.sum_distance = self.add_weight(
            name="sum_distance", initializer="zeros"
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_shape = tf.shape(y_pred)
        batch_size = y_pred_shape[0]
        logit_length = tf.fill([batch_size], y_pred_shape[1])
        decoded, _ = tf.nn.ctc_greedy_decoder(
            inputs=tf.transpose(y_pred, perm=[1, 0, 2]),
            sequence_length=logit_length,
        )
        sum_distance = tf.math.reduce_sum(tf.edit_distance(decoded[0], y_true))
        batch_size = tf.cast(batch_size, tf.float32)
        self.sum_distance.assign_add(sum_distance)
        self.total.assign_add(batch_size)

    def result(self):
        return self.sum_distance / self.total

    def reset_states(self):
        self.sum_distance.assign(0)
        self.total.assign(0)


**1. Train the model with custom dataset**

In [ ]:
!python CRNN.tf2/crnn/train.py --config mjsynth.yml --save_dir 'runs/train'

**2. Load the trained model from google drive**

In [ ]:
#Load trained model
model = keras.models.load_model(
    'model.keras',
    custom_objects={'CTCLoss': CTCLoss, 'SequenceAccuracy': SequenceAccuracy}
)

**3. Predict with trained model**

In [ ]:
def clean_string(string):
  cleaned_string = ''
  new_check = False
  i = 0
  while i < len(string):
    # print(f"i: {i}")
    if i == len(string) - 1 and string[i] != string[i-1] and string[i] != '|':
      cleaned_string += string[i]
      break
    elif i == len(string) - 1 and string[i] == string[i-1] and string:
      if string[i] != cleaned_string[-1] and string[i] != '|':
        cleaned_string += string[i]
        break
      elif string[i] != '|' and new_check:
        cleaned_string += string[i]
        break
      else:
        break

    if string[i] == '|':
      i += 1
      new_check = True
      continue

    if string[i] != string[i+1]:
      # print(string[i])
      cleaned_string += string[i]
      i += 1
      new_check = False
      continue

    if string[i] == string[i+1]:
      i += 1
      continue
  return cleaned_string

def translate_predict(pred):
  string = ''
  cleaned_string = ''
  # print(pred[0])
  for result in pred[0]:
    digit = str(np.argmax(result))
    if digit == '10':
      string += '|'
      continue
    string += str(np.argmax(result))
  print(string)
  cleaned_string = clean_string(string)
  return cleaned_string

def predict(image_path):
  image = cv2.imread(image_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  cv2_imshow(image)
  height_ratio = image.shape[0]/32
  image = cv2.resize(image, (int(image.shape[1]/ratio), 32))
  image = image.reshape(1, image.shape[0], image.shape[1], 3)
  image = image / 255.0
  pred = model.predict(image)
  return pred

In [ ]:
image_path = cv2.imread('/content/drive/MyDrive/CCCD/data_for_prediction/new_test_data_ids5/811.jpg')

pred = predict(image_path)
print(f"Model prediction: {translate_predict(pred)}")